### Load the API keys

In [48]:
import os
from dotenv import load_dotenv
load_dotenv('cred.env')
cohere_api_key = os.environ.get("COHERE_API_KEY")
replicate_api_key = os.environ.get("REPLICATE_API_TOKEN")

In [49]:
import replicate

In [54]:
!curl https://api.replicate.com/v1/account -H "Authorization: Bearer $REPLICATE_API_TOKEN"


{"type":"user","username":"snr16","name":"Niranjan Reddy","github_url":"https://github.com/snr16"}

In [55]:
replicate.run('meta/meta-llama-3-8b',input={'prompt':'write a poem'})

ReplicateError: ReplicateError Details:
title: Unauthenticated
status: 401
detail: You did not pass an authentication token

### Read data from file directory

In [142]:
from llama_index.core import SimpleDirectoryReader

# load data
loader = SimpleDirectoryReader(
            input_dir = '/Users/niranjan/Documents/data',
            recursive=True
        )
docs = loader.load_data()

### Create Embeddings

In [143]:
from llama_index.embeddings.cohere import CohereEmbedding

embed_model = CohereEmbedding(
    cohere_api_key=api_key,
    model_name="embed-english-v3.0",
    input_type="search_query",
)

In [144]:
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex

# ====== Create vector store and upload indexed data ======
Settings.embed_model = embed_model # we specify the embedding model to be used
index = VectorStoreIndex.from_documents(docs)

### Create reranker

In [145]:
from llama_index.llms.cohere import Cohere
from llama_index.postprocessor.cohere_rerank import CohereRerank

In [146]:
from llama_index.llms.ollama import Ollama

ollama_llm = Ollama(model="gemma:2b", request_timeout=60.0)
cohere_llm = Cohere(api_key=api_key)
cohere_rerank = CohereRerank(api_key=api_key)


In [147]:
def generate_prompt(file_contents='', query=''):
    from llama_index.core import PromptTemplate

    # Modify the prompt template to include file contents and the query
 
    qa_prompt_tmpl_str = (
            "Context information is below.\n"
            "---------------------\n"
            "{file_contents}\n"
            "---------------------\n"
            "Given the context information above I want you to think step by step to answer the query in a crisp manner, incase case you don't know the answer say 'I don't know!'.\n"
            "Query: {query_str}\n"
            "Answer: "
            )

    # Create a PromptTemplate object with the modified prompt template
    qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

    # Format the template with the provided file_contents and query
    prompt = qa_prompt_tmpl.partial_format(file_contents=file_contents, query=query)

    return prompt


In [148]:
import os
import time
import io
from PIL import Image
import uuid
import base64
import PyPDF2

In [149]:
# import panel as pn
# from panel.widgets import FileInput,TextInput,TextAreaInput
# from panel.chat import ChatMessage,ChatInterface
# pn.extension('plotly',template='fast')


# file_ = ''

# # Function to extract text from a PDF file
# def extract_text_from_pdf(file_contents):
#     text = ""
#     pdf_reader = PyPDF2.PdfReader(file_contents)
#     for page_num in range(len(pdf_reader.pages)):
#         page = pdf_reader.pages[page_num]
#         text += page.extract_text()
#     return text

# def upload_file(content):

#     if not isinstance(content, str):

#         spinner.value = True
#         content = io.BytesIO(content)
#         contents = extract_text_from_pdf(content)
#     else:
#         contents = content

#     # Limit content length
#     if len(contents) < 1000:
#         contents = contents
#     else:
#         contents = 'Hey! Number of tokens should not exceed 1000, try asking a small question'

#     spinner.value = False

#     return contents


# async def callback(contents, user, instance):
#     print('user',user)
    
#     if user == 'Student':
#         # await asyncio.sleep(1.8)
#         message = ''
#         if file_ == 'upload' and watcher_contents.value == '':
#             chat_message = ChatMessage('File Processing failed! Retry again',user='bot',avatar='bot_img.png')
#             yield chat_message

#         elif file_ == 'upload':
#             layout[0][2][1][1] = upload_button

#         file_contents = watcher_contents.value
        
#         if isinstance(file_contents, list):
#             content = ''
#             for c in file_contents:
#                 content+=upload_file(c)

#             file_contents = content
#         elif not file_contents == '':
#             file_contents = upload_file(file_contents)
#         else:
#             pass
        
#         if 'exceed' in file_contents:
#             chat_message = ChatMessage('Hey! Number of tokens should not exceed 1000, try asking a small question',user='bot',avatar='bot_img.png')
#             yield chat_message
#         else:
#             # Generate a prompt for question answering
#             qa_prompt_tmpl = generate_prompt(file_contents=file_contents)
#             query_engine.update_prompts({"response_synthesizer:text_qa_template": qa_prompt_tmpl})

#             # Query the generated response
#             cohere_response = query_engine.query(contents)
#             msg =''
#             for word in cohere_response.response_gen:
#                 msg+=word
#                 yield ChatMessage(msg,user="Cohere",avatar='cohere_img.png')

#             ############################## Ollama ##############################

#             #Ollama llm
#             Settings.llm = ollama_llm
#             # Query the generated response
#             ollama_response = query_engine.query(contents)
#             msg =''
#             for word in cohere_response.response_gen:
#                 msg+=word
#                 yield ChatMessage(msg,user='Ollama',avatar='ollama_img.png')




# def on_upload_click(event):
    
#     # Initialize new file input
#     new_file_input = pn.widgets.FileInput(accept=".pdf",multiple=True,align='start',width_policy='fit')
    
#     # Link new file input to watcher_contents
#     new_file_input.link(watcher_contents, value="value")

#     # Update layout with the new file input
#     layout[0][2][1][1] = new_file_input

#     global file_
#     file_ = 'upload'


# def decode_image(image_path):
#     # Read the image content
#     with open(image_path, "rb") as f:
#         image_content = f.read()


#     # Encode the image content to Base64
#     image_base64 = base64.b64encode(image_content).decode()

#     return image_base64



# # Initialize watcher_contents
# watcher_contents = pn.widgets.StaticText(value="")

# # Initialize spinner
# spinner = pn.indicators.LoadingSpinner(value=False, width=30, height=30)

# upload_button = pn.widgets.Button(name='🔗')
# upload_button.on_click(on_upload_click)

# text_path = "text_img.png"
# text_base64 = decode_image(text_path)



# introduction_text = """

# <div style="font-size: 32px; text-align: center; color: #F5F5F5; background-color: #5F9EA0; padding: 20px; width: 400px; border-radius: 2px 2px 5px 5px; border: 2px solid #4682B4; box-shadow: 0px 0px 10px 0px rgba(0, 0, 0, 0.5);">
#         <b>Welcome to our Chat Interface</b>
# </div>


# <div style="font-size: 24px; text-align: center; width: 400px;padding:15px">
#     This chat interface allows you to interact with our AI assistant <img src="data:image/png;base64,{image_base64}" style="width: 30px; height: 30px; vertical-align: -5px;">.
#     Feel free to ask any questions you have!
# </div>
# """.format(image_base64=text_base64)

# # Define the image
# image_path = 'cohere.png'
# image_pane = pn.panel(image_path, width=200)



# # Define the chat interface
# chat_interface = ChatInterface(
#     user='Student',
#     avatar='user_img.png',
#     callback=callback,
#     callback_user='Cohere',
#     callback_exception='verbose',
#     sizing_mode="stretch_width",
#     reaction_icons={"like": "thumb-up", "dislike": "thumb-down"},
#     default_avatars={"student": 'user_img.png', "Cohere": 'cohere_img.png','Ollama':'ollama_img.png'},
#     widgets=[TextInput(placeholder='Ask me Question!',description='This is chatbot developed to answer questions from class docs')],
# )

# #  Initial message
# initial_message = chat_interface.send("Hey! Ask me a question",user='bot',respond='False',avatar='bot_img.png')

# # Load your local image file
# background_path = "background_img.png"
# background_base64 = decode_image(background_path)



# # Combine everything into a layout
# layout = pn.Column(  # Set the background image
#     pn.Row(
#         pn.Spacer(width=50),  
#         pn.Column(
#             pn.Spacer(height=120),
#             pn.panel(introduction_text, width=400, height=300, sizing_mode='fixed', styles={'background': '#98FB98', 'color': 'black'}),
#             pn.Row(pn.Spacer(width=100),
#             # image_pane
#             ),
#             width=400,  
#         ),
#         pn.Row(
#         pn.Spacer(width=50),
#         pn.Column(pn.Spacer(height=902),
#         upload_button),margin=(0,10,0,10)),
#         pn.Row(
#         chat_interface,
#         sizing_mode='scale_width',
#         )
#     ),
#     sizing_mode="stretch_width",
#     styles={
#         'background-image': f'url(data:image/png;base64,{background_base64}),linear-gradient(to left, #D8BFD8 , #87CEEB)',
#         'background-size': '30%,cover',
#         'background-repeat': 'no-repeat',
#         'background-attachment': 'fixed',
#         'background-position': 'center',
#     } 
# )

# layout.show()




In [150]:
import panel as pn
from panel.widgets import FileInput,TextInput,TextAreaInput
from panel.chat import ChatMessage,ChatInterface
pn.extension('plotly',template='fast')


file_ = ''

# Function to extract text from a PDF file
def extract_text_from_pdf(file_contents):
    text = ""
    pdf_reader = PyPDF2.PdfReader(file_contents)
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text += page.extract_text()
    return text

def upload_file(content):

    if not isinstance(content, str):

        spinner.value = True
        content = io.BytesIO(content)
        contents = extract_text_from_pdf(content)
    else:
        contents = content

    # Limit content length
    if len(contents) < 1000:
        contents = contents
    else:
        contents = 'Hey! Number of tokens should not exceed 1000, try asking a small question'

    spinner.value = False

    return contents


async def callback(contents, user, instance):
    
    if user == 'Student':
        # await asyncio.sleep(1.8)
        message = ''
        if file_ == 'upload' and watcher_contents.value == '':
            chat_message = ChatMessage('File Processing failed! Retry again',user='bot',avatar='bot_img.png')
            yield chat_message

        elif file_ == 'upload':
            layout[0][2][1][1] = upload_button

        file_contents = watcher_contents.value
        
        if isinstance(file_contents, list):
            content = ''
            for c in file_contents:
                content+=upload_file(c)

            file_contents = content
        elif not file_contents == '':
            file_contents = upload_file(file_contents)
        else:
            pass
        
        if 'exceed' in file_contents:
            chat_message = ChatMessage('Hey! Number of tokens should not exceed 1000, try asking a small question',user='bot',avatar='bot_img.png')
            yield chat_message
        else:
            # Generate a prompt for question answering
            query_engine = index.as_query_engine(llm = cohere_llm,streaming=True,nodeprocessor=[cohere_rerank])
            qa_prompt_tmpl = generate_prompt(file_contents=file_contents)
            query_engine.update_prompts({"response_synthesizer:text_qa_template": qa_prompt_tmpl})

            # Query the generated response
            cohere_response = query_engine.query(contents)


            response = [value for value in cohere_response.response_gen]
            msg = contents +'???\n\n'
            for i,word in enumerate(response):
                msg+=word
                yield ChatMessage(msg,user='Cohere',avatar='cohere_img.png')

            instance.respond()

    elif user == 'Cohere':
        (query,contents) = tuple(contents.split('???'))

        #Ollama llm
        # Query the generated response
        query_engine = index.as_query_engine(llm = ollama_llm,streaming=True,nodeprocessor=[cohere_rerank])
        ollama_response = query_engine.query(query)
        
        response = [value for value in ollama_response.response_gen]
        msg = query +'???\n\n' 
        for i,word in enumerate(response):
            msg+=word
            yield ChatMessage(msg,user='Ollama',avatar='ollama_img.png')

        instance.respond()

    elif user == 'Ollama':
        instance.send('Completed',respond=False)
        
    # else:
    #     yield ''


def on_upload_click(event):
    
    # Initialize new file input
    new_file_input = pn.widgets.FileInput(accept=".pdf",multiple=True,align='start',width_policy='fit')
    
    # Link new file input to watcher_contents
    new_file_input.link(watcher_contents, value="value")

    # Update layout with the new file input
    layout[0][2][1][1] = new_file_input

    global file_
    file_ = 'upload'


def decode_image(image_path):
    # Read the image content
    with open(image_path, "rb") as f:
        image_content = f.read()


    # Encode the image content to Base64
    image_base64 = base64.b64encode(image_content).decode()

    return image_base64



# Initialize watcher_contents
watcher_contents = pn.widgets.StaticText(value="")

# Initialize spinner
spinner = pn.indicators.LoadingSpinner(value=False, width=30, height=30)

upload_button = pn.widgets.Button(name='🔗')
upload_button.on_click(on_upload_click)

text_path = "text_img.png"
text_base64 = decode_image(text_path)



introduction_text = """

<div style="font-size: 32px; text-align: center; color: #F5F5F5; background-color: #5F9EA0; padding: 20px; width: 400px; border-radius: 2px 2px 5px 5px; border: 2px solid #4682B4; box-shadow: 0px 0px 10px 0px rgba(0, 0, 0, 0.5);">
        <b>Welcome to our Chat Interface</b>
</div>


<div style="font-size: 24px; text-align: center; width: 400px;padding:15px">
    This chat interface allows you to interact with our AI assistant <img src="data:image/png;base64,{image_base64}" style="width: 30px; height: 30px; vertical-align: -5px;">.
    Feel free to ask any questions you have!
</div>
""".format(image_base64=text_base64)

# Define the image
image_path = 'cohere.png'
image_pane = pn.panel(image_path, width=200)



# Define the chat interface
chat_interface = ChatInterface(
    user='Student',
    avatar='user_img.png',
    callback=callback,
    callback_user='Cohere',
    callback_exception='verbose',
    sizing_mode="stretch_width",
    reaction_icons={"like": "thumb-up", "dislike": "thumb-down"},
    default_avatars={"student": 'user_img.png', "Cohere": 'cohere_img.png','Ollama':'ollama_img.png'},
    widgets=[TextInput(placeholder='Ask me Question!',description='This is chatbot developed to answer questions from class docs')],
)

#  Initial message
initial_message = chat_interface.send("Hey! Ask me a question",user='bot',respond='False',avatar='bot_img.png')

# Load your local image file
background_path = "background_img.png"
background_base64 = decode_image(background_path)



# Combine everything into a layout
layout = pn.Column(  # Set the background image
    pn.Row(
        pn.Spacer(width=50),  
        pn.Column(
            pn.Spacer(height=120),
            pn.panel(introduction_text, width=400, height=300, sizing_mode='fixed', styles={'background': '#98FB98', 'color': 'black'}),
            pn.Row(pn.Spacer(width=100),
            # image_pane
            ),
            width=400,  
        ),
        pn.Row(
        pn.Spacer(width=50),
        pn.Column(pn.Spacer(height=902),
        upload_button),margin=(0,10,0,10)),
        pn.Row(
        chat_interface,
        sizing_mode='scale_width',
        )
    ),
    sizing_mode="stretch_width",
    styles={
        'background-image': f'url(data:image/png;base64,{background_base64}),linear-gradient(to left, #D8BFD8 , #87CEEB)',
        'background-size': '30%,cover',
        'background-repeat': 'no-repeat',
        'background-attachment': 'fixed',
        'background-position': 'center',
    } 
)

layout.show()




Launching server at http://localhost:63065


In [151]:
def revised_generate_prompt(file_contents='', query='',bot1_answer=''):
    from llama_index.core import PromptTemplate

    # Modify the prompt template to include file contents and the query
 
    qa_prompt_tmpl_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{file_contents}\n"
    "---------------------\n"
    "Your task is to listen to Bot 1's response and either provide additional information or correct any mistakes made by Bot 1. If Bot 1 is correct, acknowledge it. If Bot 1 says 'I don't know' or doesn't answer the question, provide your answer based on the context and question asked. If no context information is given, listen to Bot 1's response and correct it.\n"
    ""
    "Please note that if Bot 1 asks any personal questions unrelated to the context or asks for opinions, Bot 2 should not provide additional information or corrections based on the context provided. Instead, Bot 2 should respond independently to personal questions asked to Bot 1.\n"
    ""
    "Bot 1's Answer: {bot1_answer}\n"
    "Your Additional Information/Correction:\n"
    "Examples responses for Bot 2 for personal questions\n:"
    "- Who are you: I am an AI bot\n."
    "- What is your company name: My creator is Microsoft\n."

    )

    # Create a PromptTemplate object with the modified prompt template
    qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

    # Format the template with the provided file_contents and query
    prompt = qa_prompt_tmpl.partial_format(file_contents=file_contents, query=query,bot1_answer=bot1_answer)

    return prompt


In [152]:
import panel as pn
from panel.widgets import FileInput,TextInput,TextAreaInput
from panel.chat import ChatMessage,ChatInterface
pn.extension('plotly',template='fast')


file_ = ''
file_contents = ''

# Function to extract text from a PDF file
def extract_text_from_pdf(file_contents):
    text = ""
    pdf_reader = PyPDF2.PdfReader(file_contents)
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text += page.extract_text()
    return text

def upload_file(content):

    if not isinstance(content, str):

        spinner.value = True
        content = io.BytesIO(content)
        contents = extract_text_from_pdf(content)
    else:
        contents = content

    # Limit content length
    if len(contents) < 1000:
        contents = contents
    else:
        contents = 'Hey! Number of tokens should not exceed 1000, try asking a small question'

    spinner.value = False

    return contents


async def callback(contents, user, instance):
    
    if user == 'Student':
        global file_contents
        # await asyncio.sleep(1.8)
        message = ''
        if file_ == 'upload' and watcher_contents.value == '':
            chat_message = ChatMessage('File Processing failed! Retry again',user='bot',avatar='bot_img.png')
            yield chat_message

        elif file_ == 'upload':
            layout[0][2][1][1] = upload_button

        file_contents = watcher_contents.value
        
        if isinstance(file_contents, list):
            content = ''
            for c in file_contents:
                content+=upload_file(c)

            file_contents = content
        elif not file_contents == '':
            file_contents = upload_file(file_contents)
        else:
            pass
        
        if 'exceed' in file_contents:
            chat_message = ChatMessage('Hey! Number of tokens should not exceed 1000, try asking a small question',user='bot',avatar='bot_img.png')
            yield chat_message
        else:
            #intialize the query engine for cohere_llm
            query_engine = index.as_query_engine(llm = ollama_llm,streaming=True,nodeprocessor=[cohere_rerank])
            # Generate a prompt for question answering
            qa_prompt_tmpl = generate_prompt(file_contents=file_contents)
            query_engine.update_prompts({"response_synthesizer:text_qa_template": qa_prompt_tmpl})

            # Query the generated response
            cohere_response = query_engine.query(contents)


            response = [value for value in cohere_response.response_gen]
            msg = ''
            for i,word in enumerate(response):
                msg+=word
                yield ChatMessage(msg,user='Cohere',avatar='cohere_img.png')

            instance.respond()

    elif user == 'Cohere':

        #Ollama llm
        query_engine = index.as_query_engine(llm = cohere_llm,streaming=True,nodeprocessor=[cohere_rerank])
        qa_prompt_tmpl = revised_generate_prompt(file_contents=file_contents,bot1_answer=contents)
        query_engine.update_prompts({"response_synthesizer:text_qa_template": qa_prompt_tmpl})
        # Query the generated response
        ollama_response = query_engine.query(contents)
        
        response = [value for value in ollama_response.response_gen]
        msg = '' 
        for i,word in enumerate(response):
            msg+=word
            yield ChatMessage(msg,user='Ollama',avatar='ollama_img.png')

        instance.respond()

    elif user == 'Ollama':
        instance.send('Completed',respond=False)
        
    # else:
    #     yield ''


def on_upload_click(event):
    
    # Initialize new file input
    new_file_input = pn.widgets.FileInput(accept=".pdf",multiple=True,align='start',width_policy='fit')
    
    # Link new file input to watcher_contents
    new_file_input.link(watcher_contents, value="value")

    # Update layout with the new file input
    layout[0][2][1][1] = new_file_input

    global file_
    file_ = 'upload'


def decode_image(image_path):
    # Read the image content
    with open(image_path, "rb") as f:
        image_content = f.read()


    # Encode the image content to Base64
    image_base64 = base64.b64encode(image_content).decode()

    return image_base64



# Initialize watcher_contents
watcher_contents = pn.widgets.StaticText(value="")

# Initialize spinner
spinner = pn.indicators.LoadingSpinner(value=False, width=30, height=30)

upload_button = pn.widgets.Button(name='🔗')
upload_button.on_click(on_upload_click)

text_path = "text_img.png"
text_base64 = decode_image(text_path)



introduction_text = """

<div style="font-size: 32px; text-align: center; color: #F5F5F5; background-color: #5F9EA0; padding: 20px; width: 400px; border-radius: 2px 2px 5px 5px; border: 2px solid #4682B4; box-shadow: 0px 0px 10px 0px rgba(0, 0, 0, 0.5);">
        <b>Welcome to our Chat Interface</b>
</div>


<div style="font-size: 24px; text-align: center; width: 400px;padding:15px">
    This chat interface allows you to interact with our AI assistant <img src="data:image/png;base64,{image_base64}" style="width: 30px; height: 30px; vertical-align: -5px;">.
    Feel free to ask any questions you have!
</div>
""".format(image_base64=text_base64)

# Define the image
image_path = 'cohere.png'
image_pane = pn.panel(image_path, width=200)



# Define the chat interface
chat_interface = ChatInterface(
    user='Student',
    avatar='user_img.png',
    callback=callback,
    callback_user='Cohere',
    callback_exception='verbose',
    sizing_mode="stretch_width",
    reaction_icons={"like": "thumb-up", "dislike": "thumb-down"},
    default_avatars={"student": 'user_img.png', "Cohere": 'cohere_img.png','Ollama':'ollama_img.png'},
    widgets=[TextInput(placeholder='Ask me Question!',description='This is chatbot developed to answer questions from class docs')],
)

#  Initial message
initial_message = chat_interface.send("Hey! Ask me a question",user='bot',respond='False',avatar='bot_img.png')

# Load your local image file
background_path = "background_img.png"
background_base64 = decode_image(background_path)



# Combine everything into a layout
layout = pn.Column(  # Set the background image
    pn.Row(
        pn.Spacer(width=50),  
        pn.Column(
            pn.Spacer(height=120),
            pn.panel(introduction_text, width=400, height=300, sizing_mode='fixed', styles={'background': '#98FB98', 'color': 'black'}),
            pn.Row(pn.Spacer(width=100),
            # image_pane
            ),
            width=400,  
        ),
        pn.Row(
        pn.Spacer(width=50),
        pn.Column(pn.Spacer(height=902),
        upload_button),margin=(0,10,0,10)),
        pn.Row(
        chat_interface,
        sizing_mode='scale_width',
        )
    ),
    sizing_mode="stretch_width",
    styles={
        'background-image': f'url(data:image/png;base64,{background_base64}),linear-gradient(to left, #D8BFD8 , #87CEEB)',
        'background-size': '30%,cover',
        'background-repeat': 'no-repeat',
        'background-attachment': 'fixed',
        'background-position': 'center',
    } 
)

layout.show()




Launching server at http://localhost:63068


In [153]:
ollama_llm

Ollama(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x15f2ad410>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x12b9a74c0>, completion_to_prompt=<function default_completion_to_prompt at 0x12c00b100>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, base_url='http://localhost:11434', model='gemma:2b', temperature=0.75, context_window=3900, request_timeout=60.0, prompt_key='prompt', additional_kwargs={})

In [154]:
Settings.llm = ollama_llm

In [155]:
print(query_engine.query('What is your company name'))

The context does not provide any information about the company name, so I cannot answer this query from the provided context.


In [156]:
query_engine.query('What is your company name')

Response(response='The context does not provide any information about the company name, so I cannot answer this query from the provided context.', source_nodes=[NodeWithScore(node=TextNode(id_='8f1430ad-cb93-4523-9e3a-84268bbae8c9', embedding=None, metadata={'file_path': '/Users/niranjan/Documents/data/creds.env', 'file_name': 'creds.env', 'file_size': 48, 'creation_date': '2024-04-26', 'last_modified_date': '2024-04-26'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='b7c32faa-ea4b-4654-8185-4e94d49675a7', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/Users/niranjan/Documents/data/creds.env', 'file_name': 'creds.env', 'file_size': 48, 'creation_date': '2024-04-26', 'last_modified_date': '

In [157]:
ollama_llm

Ollama(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x14df13390>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x12b9a74c0>, completion_to_prompt=<function default_completion_to_prompt at 0x12c00b100>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, base_url='http://localhost:11434', model='gemma:2b', temperature=0.75, context_window=3900, request_timeout=60.0, prompt_key='prompt', additional_kwargs={})